# Imports

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline
from tqdm import tqdm
from typing import List,Dict
from IPython.display import Image
from IPython.core.display import HTML 
from pathlib import Path

In [4]:
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download("stopwords")
nltk.download("punkt")
from string import punctuation, ascii_lowercase
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\omer.l\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\omer.l\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Debug


In [5]:
DEBUG = False

N_ROWS_FOR_DEBUG = 5*10**3 

# Download Data
`Download the data` from [HERE](https://github.com/ludovicaschaerf/TMCI_Project/raw/master/data/380000-lyrics-from-metrolyrics.zip) and put it in the directory the script is.

# Config

In [6]:
INPUT_FILE_PATH = Path("lyrics.csv")
BOW_PATH = Path("bow.csv")
N_ROWS = N_ROWS_FOR_DEBUG if DEBUG else None
CHUNCK_SIZE = 5 if DEBUG else 5*10**3
tqdm_n_iterations = N_ROWS//CHUNCK_SIZE +1 if DEBUG else 363*10**3//CHUNCK_SIZE + 1
COLS = [5]

## 1.1 Bag of words /TfIdf model
### Implement the following methods:

* `preprocess_sentence`: 
    * Lower case the word
    * Ignores it if it's in the stopwords list
    * Removes characters which are not in the allowed symbols
    * Stems it and appends it to the output sentence
    * Discards words with length <= 1
    
    
* `update_counts_and_probabilities`: 

    * Update self.unigram count (the amount of time each word is in the text)
    * Update self.bigram count (two consecutive word occurances)
    * Update self.trigram count (three consecutive word occurances)
    * Update inverted index: a dictionary with words as keys and the values is a dictionary - {'DocID' : word_count}   
    
* `compute_word_document_frequency`:

   * For each word count the number of docs it appears in. For example , for the word 'apple' -
$$\sum_{i \in docs} I(apple \in doc_i), I := Indicator function$$


* `update_inverted_index_with_tf_idf_and_compute_document_norm`:

    * Update the inverted index (which currently hold word counts) with tf idf weighing. We will compute tf by dividing with the number of words in each document. 
    * As we want to calculate the document norm, incrementally update the document norm. pay attention that later we apply sqrt to it to finish the process.

#### The result of this code is a bag of words model that already counts for TF-IDF weighing

In [7]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
allowed_symbols = set(l for l in ascii_lowercase)

In [8]:
def preprocess_sentence(sentence : str) -> List[str]:
    output_sentence = []
    for word in word_tokenize(sentence):
        word = word.lower()
        word = ''.join([i for i in word if i in allowed_symbols])
        if(word in stop_words):
            continue  
        word = stemmer.stem(word)
        if(len(word)>1):
            output_sentence.append(word)
    return output_sentence
    

def get_data_chuncks() -> List[str]:
    for i ,chunck in enumerate(pd.read_csv(INPUT_FILE_PATH, usecols = COLS, chunksize = CHUNCK_SIZE, nrows = N_ROWS)):
        chunck = chunck.values.tolist()
        yield [chunck[i][0] for i in range(len(chunck))] 

class TfIdf:
    def __init__(self):
        self.unigram_count =  Counter()
        self.bigram_count = Counter()
        self.trigram_count = Counter()
        self.document_term_frequency = Counter()
        self.word_document_frequency = {}
        self.inverted_index = {}
        self.doc_norms = {}
        self.n_docs = -1
        self.sentence_preprocesser = preprocess_sentence
        self.bow_path = BOW_PATH

    def update_counts_and_probabilities(self, sentence :List[str],document_id:int) -> None:
        sentence_len = len(sentence)
        self.document_term_frequency[document_id] = sentence_len
        for i,word in enumerate(sentence):
            self.unigram_count[word] += 1 
            if(i < len(sentence)-1):
                self.bigram_count[(sentence[i],sentence[i+1])] += 1
            if(i < len(sentence)-2):
                self.trigram_count[(sentence[i],sentence[i+1],sentence[i+2])] += 1
            if word not in self.inverted_index:
                self.inverted_index.update({word:{document_id:1}})
            else:
                if(document_id in self.inverted_index[word]):
                    self.inverted_index[word][document_id] += 1
                else:
                    self.inverted_index[word].update({document_id: 1}) 
                    
    def fit(self) -> None:
        for chunck in tqdm(get_data_chuncks(), total = tqdm_n_iterations):
            for sentence in chunck:
                self.n_docs += 1 
                if not isinstance(sentence, str):
                    continue
                sentence = self.sentence_preprocesser(sentence)
                if sentence:
                    self.update_counts_and_probabilities(sentence,self.n_docs)
        self.save_bow() # bow is 'bag of words'
        self.compute_word_document_frequency()
        self.update_inverted_index_with_tf_idf_and_compute_document_norm()
             
    def compute_word_document_frequency(self):
        for word in self.inverted_index.keys():
            self.word_document_frequency[word] = len(self.inverted_index[word])
            
    def update_inverted_index_with_tf_idf_and_compute_document_norm(self):
        for term in self.inverted_index:
            for doc, freq in self.inverted_index[term].items():
                self.inverted_index[term][doc] = (freq / self.document_term_frequency[doc] * np.log10(self.n_docs/self.word_document_frequency[term]))
                if doc not in self.doc_norms:
                    self.doc_norms.update({doc:0}) 
                self.doc_norms[doc] += (self.inverted_index[term][doc]**2)
        for doc in self.doc_norms.keys():
            self.doc_norms[doc] = np.sqrt(self.doc_norms[doc]) 
            
    def save_bow(self):
        pd.DataFrame([self.inverted_index]).T.to_csv(self.bow_path)
                
tf_idf = TfIdf()
tf_idf.fit()

100%|██████████████████████████████████████████████████████████████████████████████████| 73/73 [17:48<00:00, 14.64s/it]


## 1.11 Bag of words model:

1. What is the computational complexity of this model, as a function of the number of docs in the corpus?
2. How can we make this code better in terms running time (parallelization or other topics you find)? 

## 1.2 DocumentRetriever
Not this retriever &#8595;


![dsafdsafsdafdsf](https://cdn3-www.dogtime.com/assets/uploads/2019/10/golden-cocker-retriever-mixed-dog-breed-pictures-cover-1.jpg)

In [10]:
class DocumentRetriever:
    def __init__(self, tf_idf):
        self.sentence_preprocesser = preprocess_sentence
        self.vocab = set(tf_idf.unigram_count.keys())
        self.n_docs = tf_idf.n_docs
        self.inverted_index = tf_idf.inverted_index
        self.word_document_frequency = tf_idf.word_document_frequency
        self.doc_norms = tf_idf.doc_norms

    def rank(self, query: Dict[str, int], documents: Dict[str, Counter], metric: str) -> Dict[str, float]:
        result = {}  # key: DocID , value : float , simmilarity to query
        query_len = np.sum(np.array(list(query.values())))
        for term, count in query.items(): #in this loop we're updating the query's weights 
            query[term] = (count / query_len * np.log10(tf_idf.n_docs / tf_idf.word_document_frequency[term]))
            for doc, freq in documents[term].items():
                if (doc not in result):
                    result.update({doc: 0})
                if metric == 'inner_product':
                    result[doc] += query[term] * freq
                if metric == 'cosine':
                    result[doc] += (query[term] * freq / self.doc_norms[doc])

        return result

    def sort_and_retrieve_k_best(self, scores: Dict[str, float], k: int):
        ### YOUR CODE HERE
        return list({k: v for k, v in sorted(scores.items(), key=lambda item: item[1],reverse=True)})[:k]
        ### END YOUR CODE

    def reduce_query_to_counts(self, query: List) :
        return dict(Counter(query)) # rank get Dict as input so we used this cast (even that a counter is a dict)

    def get_top_k_documents(self, query: str, metric: str, k=5) -> List[str]:
        query = self.sentence_preprocesser(query)
        query = [word for word in query if word in self.vocab]  # filter nan
        query_bow = self.reduce_query_to_counts(query)
        relavant_documents = {word: self.inverted_index.get(word) for word in query}
        ducuments_with_similarity = self.rank(query_bow, relavant_documents, metric)
        return self.sort_and_retrieve_k_best(ducuments_with_similarity, k)
        
dr = DocumentRetriever(tf_idf)

In [11]:
from IPython.display import HTML
query = "Better stop dreaming of the quiet life, 'cause it's the one we'll never know And quit running for that runaway bus 'cause those rosy days are few And stop apologizing for the things you've never done 'Cause time is short and life is cruel but it's up to us to change This town called malice"
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/KT6ZtUbVw1M?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

c:\users\omer.l\appdata\local\programs\python\python37\lib\site-packages\IPython\core\display.py:689: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [12]:
cosine_top_k = dr.get_top_k_documents(query, 'cosine')
print(cosine_top_k)
inner_product_top_k = dr.get_top_k_documents(query, 'inner_product')
print(inner_product_top_k)

[66192, 98277, 318295, 234637, 107511]
[66192, 61393, 128160, 309297, 318295]


In [13]:
for index, song in enumerate(pd.read_csv(INPUT_FILE_PATH,usecols = [5]).iloc[cosine_top_k]['lyrics']):
    sep = "#"*50
    print(F"{sep}\nsong #{index} \n{song} \n{sep}")

##################################################
song #0 
I hear people talk
I see people walk
They seem so out of touch
I wanna get away so much
All the clockwork toys
Are making too much noise
It's the machinery
It's breaking down, oh can't you see
Run runaway, run runaway
Run runaway with me
Run runaway, run runaway
Run runaway with me
The picture doesn't change
It's just a frozen frame
I wanna break the ice
I wanna go to paradise
There is nowhere to hide
I'll take you for a ride
But not if you kiss and tell
I don't mean on a carousel
Won't you run runaway, run runaway
Run runaway with me
Run runaway, run runaway
Run runaway with me
One day I'm to say my
Three wishes came true
Till then I pretend I'm
Escaping with you, you, you, you
Run runaway, run runaway
Run runaway with me
Run runaway, run runaway
Run runaway with me
Run runaway, run runaway
Run runaway with me
Run runaway, run runaway
Run runaway with me
Run runaway, run runaway
Run runaway with me
Run runaway, run runaway
Ru

# 1.3 term statistics:
Use "bow" object that we created earlier and answer the following questions:

1. How many unique words we have?
2. How many potential word bigrams we have? How many actual word bigrams we have? How do you explain this difference?
3. What is the storage size of the input file "lyrics.csv"? What is the output file (bow.csv) size? how do you explain this difference?  

In [14]:
# 1. 
df = pd.read_csv(BOW_PATH)
print(df.shape[0])

"""
### Your verbal solution here
we have 388243 unique words!
### End your verbal solution here
"""

# 2.
print(len(tf_idf.bigram_count))

"""
### Your verbal solution here
we could potentialy have n^2 words but we took only the consecutive (and not all the possible pair of the words). this is why we ended up with 
a number than is not enormous.

### End your verbal solution here
"""

# 3.
print('Original file (lyrics.csv) size is {} bytes, and Bow file size is {} bytes'.format(INPUT_FILE_PATH.stat().st_size,BOW_PATH.stat().st_size))

"""
### Your verbal solution here

Original file (lyrics.csv) size is 324632382 bytes, and Bow file size is 193319838 bytes- we managed to decrease the file size in about 40%
because we cleaned up repetitive words and many other words that we omitted in the sentece processing phase. we ended up with smaller and 
(hopefully) more informative data.

### End your verbal solution here
"""

388243
6312785
Original file (lyrics.csv) size is 324632382 bytes, and Bow file size is 193319838 bytes


'\n### Your verbal solution here\n\nOriginal file (lyrics.csv) size is 324632382 bytes, and Bow file size is 193319838 bytes- we managed to decrease the file size in about 40%\nbecause we cleaned up repetitive words and many other words that we omitted in the sentece processing phase. we ended up with smaller and \n(hopefully) more informative data.\n\n### End your verbal solution here\n'

## 1.4 NgramSpellingCorrector
Now we will implement a Ngarm (character Ngrams) spelling corrector. That is, we have an out of vocabulary word (w) and we want to retrieve the most similar words (in our vocabulary) to this word.
we will model the similarity of two words by-

$$sim(v,w) := prior \cdot likelihood = p(w) \cdot P(v|w) $$ 
$$P(v|w) := JaccardIndex =  \frac{|X \cap Y|}{|X \cup Y|}$$

Where v is an out of vocabulary word (typo or spelling mistake), w is in a vocabulary word, X is the ngram set of v and Y is the ngram set of w.
For example, if n == 3, the set of ngrams for word "banana" is set("ban","ana","nan","ana") = {"ban","ana","nan"}

In order to do it efficently, we will first construct an index from the possible Ngrams we have seen in our corpus to the words that those Ngrams appear in, in order prevent comparing w to all of the words in our corpus.
Then, we will implement a function that computes this similarity.

* Make sure you compute the JaccardIndex efficently!

In [15]:
def get_bigrams(word):
    for ngram in nltk.ngrams(word, 2):
        yield "".join(list(ngram))
    
def get_trigrams(word):
    for ngram in nltk.ngrams(word, 3):
        yield "".join(list(ngram))

In [16]:
class NgramSpellingCorrector:
    def __init__(self, unigram_counts: Counter, get_n_gram: callable):
        self.unigram_counts = unigram_counts
        self.ngram_index = {}
        self.get_n_grams = get_n_gram
    
    def build_index(self) -> None:
        for word in self.unigram_counts:
            for ngram in self.get_n_grams(word):
                if ngram not in self.ngram_index:
                    self.ngram_index[ngram] = set()
                self.ngram_index[ngram].add(word)
        
    def get_top_k_words(self,word:str,k=5) -> List[str]:
        relevant_ngrams = []
        for ngram in self.get_n_grams(word):
            relevant_ngrams.append(ngram)
        list_of_words = []
        for words in [self.ngram_index[x] for x in set(self.ngram_index.keys()).intersection(set(relevant_ngrams))]:
            for word in words:
                list_of_words.append(word)
        intersection_counter = dict(Counter(list_of_words)) # we're getting a counter of each word and the # of shared ngrams it has with the given word
        sum_unigram_values = sum(self.unigram_counts.values()) # compute it here once in order to compute p(w)
        for key_word, count in intersection_counter.items():
            intersection_counter[key_word] = (self.unigram_counts[key_word] / sum_unigram_values ) * (count / (len(word +key_word) - count))
        return list({k: v for k, v in sorted(intersection_counter.items(), key=lambda item: item[1],reverse=True)})[:k]


class BigramSpellingCorrector(NgramSpellingCorrector):
    def __init__(self, unigram_counts: Counter):
        super().__init__(unigram_counts, get_bigrams)
        
        
class TrigramSpellingCorrector(NgramSpellingCorrector):
    def __init__(self, unigram_counts: Counter):
        super().__init__(unigram_counts, get_trigrams)
        

In [17]:
out_of_vocab_word = 'supercalifragilisticexpialidocious'
bigram_spelling_corrector = BigramSpellingCorrector(tf_idf.unigram_count)
bigram_spelling_corrector.build_index()
bigram_spelling_corrector.get_top_k_words(out_of_vocab_word)

['caus', 'like', 'life', 'still', 'time']

In [18]:
trigram_spelling_corrector = TrigramSpellingCorrector(tf_idf.unigram_count)
trigram_spelling_corrector.build_index()
trigram_spelling_corrector.get_top_k_words(out_of_vocab_word)

['life', 'still', 'call', 'listen', 'hous']

## 1.5 Language model
Calculate the log likelihood of a sentence. Once with a bigram markovian langauge model, and once with a trigram model.
for example - the likelihood of the senetence "spiderman spiderman does whatever a spider can" for the bigram model is: 
$$p(spiderman)\cdot p(spiderman|spiderman) \cdot  (does|spiderman) \cdot (whatever|does) \cdot  (a|whatever) \cdot  (spider|a) \cdot (can|spider)$$

And for the trigram model:
$$p(spiderman,spiderman)\cdot p(does|spiderman,spiderman) \cdot  (whatever|spiderman,does) \cdot (a|does,whatever) \cdot  (spider|whatever,a) \cdot  (can|a, spider)$$

Since we do not want a zero probability sentence use Laplace smoothing, as you have seen in the lecture, or here https://en.wikipedia.org/wiki/Additive_smoothing

In [19]:
# for the probability smoothing
NUMERATOR_SMOOTHING = 1
DENOMINATOR_SMOOTHING = 10**4
def sentence_log_probabilty(unigrams : Counter, bigrams  : Counter,trigrams : Counter, sentence: str) :
    bigram_log_likelilhood, trigram_log_likelilhood = 0, 0
    splited_sentence = sentence.split()
    for i,word in enumerate(splited_sentence):
        bigram_d = sum([bigrams[x] for x in (y for y in bigrams.keys() if y[0] == splited_sentence[i-1])]) #denominator of p(word1|word2)- all word with contain i-1 word
        trigram_d = sum([trigrams[x] for x in (y for y in trigrams.keys() if y[0:2] == (splited_sentence[i-2] ,splited_sentence[i-1]))]) # same for trigram 
        if(i==0):
            bigram_log_likelilhood += np.log((unigrams[word] +NUMERATOR_SMOOTHING) / (sum(unigrams.values()) + DENOMINATOR_SMOOTHING))
        if (i>0):
            bigram_log_likelilhood += np.log((bigrams[(splited_sentence[i-1],splited_sentence[i])] + NUMERATOR_SMOOTHING) / (bigram_d + DENOMINATOR_SMOOTHING))
        if i==1:
            trigram_log_likelilhood += np.log((bigrams[(splited_sentence[i-1],splited_sentence[i])]+NUMERATOR_SMOOTHING) / (bigram_d +DENOMINATOR_SMOOTHING))
        if (i>1):
                trigram_log_likelilhood += np.log((trigrams[(splited_sentence[i-2],splited_sentence[i-1],splited_sentence[i])] +NUMERATOR_SMOOTHING) / (trigram_d+ DENOMINATOR_SMOOTHING))
    print(F"Bigram log likelihood is {bigram_log_likelilhood}")
    print(F"Trigram log likelihood is {trigram_log_likelilhood}")
    
sentence = "spider man spider man does whatever a spider can"
sentence_log_probabilty(tf_idf.unigram_count, tf_idf.bigram_count, tf_idf.trigram_count, sentence)

Bigram log likelihood is -80.90642826061624
Trigram log likelihood is -67.22506352284744


## 1.51 Language model: B
For each model what is the next word prediciton for the sentnence "i am"?

In [9]:
# The words in "i am" have been ommited in the sentence processing phase. therefore, we won't see the words 'i' or 'am' in our unigram and nor 
# in the bigram or trigram. what would make sense to do in this case is maybe to get the word with the highest likelihood in the corpus.
# for the generic case of next word prediction given 2 other words we can compute the p(word_3|word_1,word_2) and take the word (word_3) with the highest probability
 
filtered_dict = { key:value for (key,value) in tf_idf.trigram_count.items() if key[:2] == ('i','am')}
filtered_dict 
#  this dict is empty as we said. if it wasn't, we'd take the key with the highst count (would imply that this word has the 
              #  highest probability to apear after 'i am')
              #if we had to take a random choice, we would take the the word that appears the most in the corpus 
###

{}